In [1]:
import pandas as pd
import numpy as np
import os
from _utils import utils, transforming, filling

# 유저 특성

In [2]:
mapping_dict = utils.load_pickle('../preprocessed/mapping_dict.pickle')

In [ ]:
list(mapping_dict.keys())[:4]

In [24]:
# user_spec_filled = pd.read_csv('../preprocessed/user_spec_filled.csv')
user_spec_filled_bin = utils.load_pickle('../preprocessed/user_spec_test_bin.pickle')


In [25]:
user_spec_tr, user_spec_te = utils.split_train_test(user_spec_filled_bin, 'insert_time')
del user_spec_filled_bin, user_spec_tr

In [ ]:
user_spec_te.shape

In [16]:
submit = pd.read_csv('../BigContest_data_new/데이터분석분야_퓨처스부문_평가데이터.csv')

In [17]:
application_id_use = np.intersect1d(user_spec_te['application_id'].values, submit['application_id'].values)

In [67]:
user_spec_te_use = user_spec_te[user_spec_te['application_id'].isin(application_id_use)]

In [ ]:
user_spec_te_use.shape

In [69]:
final_user_shap = ['income_type', 'employment_type', 'purpose',
            'credit_score', 'desired_amount', 'purpose_by_income_type_SWITCHLOAN', 
            'insert_hour', 'existing_loan_amt', 'existing_loan_amt_by_credit_score_cut_quantile_first', 
            'existing_loan_cnt', 'income_type', 'desired_amount_by_income_type_std', 
            'existing_loan_amt_by_purpose_max', 'insert_day', 'gender', 'age', 'insert_month', 
            'yearly_income_by_houseown_type_mean', 'purpose_by_income_type_BUYCAR', 'employment_type']

In [70]:
final_user_shap_temp = pd.Series(final_user_shap)
final_user_shap_temp_split = final_user_shap_temp[final_user_shap_temp.str.contains('_by_')]
final_user_shap_df = final_user_shap_temp_split.str.split('_by_',expand=True).rename(columns={0:'B',1:'A'})
final_user_shap_df['A'] = final_user_shap_df['A'].str.replace('quantile_','quantile')

A = final_user_shap_df['A'].str.split('_').str[:-1]
static = final_user_shap_df['A'].str.split('_').str[-1]

final_user_shap_df['A'] = A.map(lambda x:'_'.join(x))
final_user_shap_df['static'] = static.str.replace('quantile','quantile_')

In [ ]:
final_user_shap_df

In [116]:
def make_Extracted_feature(se, A_col, B_col, static, mapping_dict = mapping_dict):
    if pd.isna(se[A_col]):
        return np.nan
    else:
        temp_map = mapping_dict[(A_col, B_col)][se[A_col]]
        return temp_map[static]

In [ ]:
# user_temp = user_spec_te_use.iloc[0]

for i in range(final_user_shap_df.shape[0]):
    final_shap_temp = final_user_shap_df.iloc[i]
    A_col = final_shap_temp['A']
    B_col = final_shap_temp['B']
    static = final_shap_temp['static']
    colname = f'{B_col}_by_{A_col}_{static}'
    f = lambda x:make_Extracted_feature(x, A_col=A_col, B_col=B_col, static=static)

    user_spec_te_use[colname] = user_spec_te_use.apply(f, axis=1)



In [ ]:
final_user_shap = ['income_type', 'employment_type', 'purpose',
            'credit_score', 'desired_amount', 'purpose_by_income_type_SWITCHLOAN', 
            'insert_hour', 'existing_loan_amt', 'existing_loan_amt_by_credit_score_cut_quantile_first', 
            'existing_loan_cnt', 'income_type', 'desired_amount_by_income_type_std', 
            'existing_loan_amt_by_purpose_max', 'insert_day', 'gender', 'age', 'insert_month', 
            'yearly_income_by_houseown_type_mean', 'purpose_by_income_type_BUYCAR', 'employment_type']
np.setdiff1d(final_user_shap,user_spec_te_use.columns.tolist())

In [ ]:
insert_time = pd.to_datetime(user_spec_te_use['insert_time'])
user_spec_te_use.loc[:,'insert_day'] = insert_time.dt.day
user_spec_te_use.loc[:,'insert_hour'] = insert_time.dt.hour
user_spec_te_use.loc[:,'insert_month'] = insert_time.dt.month


In [188]:
user_features_model = user_spec_te_use[['application_id', 'user_id']+final_user_shap]

In [ ]:
user_features_model.head()

In [ ]:
# 검증
user_spec_te_use.iloc[15]

In [ ]:
mapping_dict[('income_type','purpose')][user_spec_te_use.iloc[15]['income_type']]

In [ ]:
user_features_model.iloc[15]['purpose_by_income_type_SWITCHLOAN']

In [268]:
utils.save_pickle(user_features_model,'../preprocessed/user_features_model.pickle')

The Object saved in ../preprocessed/user_features_model.pickle.


# 상품 특성

In [2]:
mapping_dict_loan = utils.load_pickle('../preprocessed/mapping_dict_loan.pickle')

In [3]:
loan_result = pd.read_csv('../BigContest_data/loan_result.csv')
loan_result_tr, loan_result_te = utils.split_train_test(loan_result,'loanapply_insert_time')
del loan_result, loan_result_tr

In [4]:
submit = pd.read_csv('../BigContest_data_new/데이터분석분야_퓨처스부문_평가데이터.csv')

In [5]:
application_id_loan_use = np.intersect1d(loan_result_te['application_id'].values, submit['application_id'].values)
product_id_loan_use = np.intersect1d(loan_result_te['product_id'].values, submit['product_id'].values)

In [6]:
loan_result_te_use = loan_result_te[(loan_result_te['application_id'].isin(application_id_loan_use) )&(loan_result_te['product_id'].isin(product_id_loan_use))]

In [ ]:
loan_result_te.shape, loan_result_te_use.shape, submit.shape

In [8]:
final_loan_shap = ['loan_limit_by_application_id_sum', 'loan_rate', 'loan_limit_per_desired_amount',
                        'loan_rate_by_application_id_min', 'loan_limit', 'loan_rate_by_application_id_quantile_first',
                        'loan_rate_by_product_id_quantile_first', 'loan_rate_by_application_id_std', 'loan_rate_by_product_id_min',
                        'loan_rate_by_application_id_quantile_second', 'loan_rate_by_application_id_mean',
                        'loan_rate_by_application_id_max', 'desired_amount', 'loan_rate_by_bank_id_max', 'loan_rate_by_product_id_std',
                        'loan_rate_by_product_id_quantile_second', 'loan_rate_by_product_id_mean', 'loan_rate_by_product_id_quantile_third',
                        'loan_rate_by_bank_id_mean', 'loan_rate_by_application_id_quantile_third']

In [13]:
final_loan_shap_temp = pd.Series(final_loan_shap)
final_loan_shap_temp_split = final_loan_shap_temp[final_loan_shap_temp.str.contains('_by_')]
final_loan_shap_df = final_loan_shap_temp_split.str.split('_by_',expand=True).rename(columns={0:'B',1:'A'})
final_loan_shap_df['A'] = final_loan_shap_df['A'].str.replace('quantile_','quantile')

A = final_loan_shap_df['A'].str.split('_').str[:-1]
static = final_loan_shap_df['A'].str.split('_').str[-1]

final_loan_shap_df['A'] = A.map(lambda x:'_'.join(x))
final_loan_shap_df['static'] = static.str.replace('quantile','quantile_')

In [ ]:
final_loan_shap_df

In [16]:
def make_Extracted_feature(se, A_col, B_col, static, mapping_dict):
    # bank_id,product_id가 train set에 없으면서 test set에는 존재하는 경우가 있으므로 nan으로 채운다.
    if pd.isna(se[A_col]) | (se[A_col] not in mapping_dict[(A_col, B_col)].keys()):
        return np.nan
    else:
        temp_map = mapping_dict[(A_col, B_col)][se[A_col]]
        return temp_map[static]

In [ ]:
# user_temp = user_spec_te_use.iloc[0]

for i in range(final_loan_shap_df.shape[0]):
    final_shap_temp = final_loan_shap_df.iloc[i]
    A_col = final_shap_temp['A']
    if A_col == 'application_id':
        continue
    B_col = final_shap_temp['B']
    static = final_shap_temp['static']
    colname = f'{B_col}_by_{A_col}_{static}'
    f = lambda x:make_Extracted_feature(x, A_col=A_col, B_col=B_col, static=static, mapping_dict=mapping_dict_loan)

    loan_result_te_use[colname] = loan_result_te_use.apply(f, axis=1)

In [ ]:
# application_id에 따른 매칭은 test data set 내에서 통꼐값을 다시 추출한다.
loan_result_te_use_copy = loan_result_te_use.copy()
for i in range(final_loan_shap_df.shape[0]):
    
    temp_loan = final_loan_shap_df.iloc[i]
    A = temp_loan['A']
    
    if A != 'application_id':
        continue
        
    B = temp_loan['B']
    static = temp_loan['static']
    static_func = static
    if static == 'quantile_first':
        static_func = lambda x:x.quantile(0.25)
    elif static == 'quantile_second':
        static_func = 'median'
    elif static == 'quantile_third':
        static_func = lambda x:x.quantile(0.75)
        
    col_name = f"{B}_by_{A}_{static}"
    loan_result_te_static = loan_result_te_use_copy[[A,B]]
    loan_result_te_use_copy[col_name] = loan_result_te_static.groupby(A)[B].transform(static_func)
    print(col_name, ' completed')
    

In [ ]:
np.setdiff1d(final_loan_shap_temp, loan_result_te_use_copy.columns.tolist())

In [34]:
loan_result_te_use_copy = pd.merge(loan_result_te_use_copy,user_spec_te[['application_id','desired_amount']], on='application_id', how='left' )

In [ ]:
loan_result_te_use_copy['desired_amount'].isnull().sum()

In [37]:
desired_amount = loan_result_te_use_copy['desired_amount']

In [ ]:
(desired_amount==0).sum()

In [ ]:
desired_amount[desired_amount==0]=np.nan

In [ ]:
desired_amount.isnull().sum()

In [41]:
loan_result_te_use_copy['loan_limit_per_desired_amount'] = loan_result_te_use_copy['loan_limit']/desired_amount

In [ ]:
loan_result_te_use_copy['loan_limit_per_desired_amount'].isnull().sum()

In [ ]:
np.setdiff1d(final_loan_shap_temp, loan_result_te_use_copy.columns.tolist())

In [45]:
loan_features_model = loan_result_te_use_copy[['application_id','product_id'] + final_loan_shap]

In [ ]:
np.setdiff1d(final_loan_shap_temp, loan_features_model.columns.tolist())

In [47]:
utils.save_pickle(loan_features_model,'../preprocessed/loan_features_model.pickle')

The Object saved in ../preprocessed/loan_features_model.pickle.


# 두 데이터 병합

In [12]:
user_features_model = utils.load_pickle('../preprocessed/user_features_model.pickle')
loan_features_model = utils.load_pickle('../preprocessed/loan_features_model.pickle')
submit = pd.read_csv('../BigContest_data_new/데이터분석분야_퓨처스부문_평가데이터.csv')

In [ ]:
loan_features_model.head()

In [26]:
loan_features_model['bank_id'] = loan_result_te_use['bank_id'].reset_index(drop=True)

In [ ]:
user_features_model.shape, loan_features_model.shape, submit.shape

In [ ]:
np.setdiff1d(user_features_model['application_id'], loan_features_model['application_id'])

In [ ]:
np.setdiff1d(loan_features_model['application_id'],user_features_model['application_id'])

In [ ]:
user_features_model.isnull().sum()

In [ ]:
loan_features_model.isnull().sum()

In [41]:
testSet = pd.merge(user_features_model, loan_features_model, on='application_id', how='right')

In [ ]:
testSet.shape

In [ ]:
# desired_amount_x, desired_amount_y는 0을 nan으로 처리하여 나누기 에러를 없애며 생긴 차이이므로 desired_amount_x만 남기면 된다.
testSet.isnull().sum()

In [47]:
testSet.drop('desired_amount_y',axis=1,inplace=True)
testSet.rename(columns = {'desired_amount_x':'desired_amount'},inplace=True)

In [48]:
utils.save_pickle(testSet, '../preprocessed/testSet.pickle')

The Object saved in ../preprocessed/testSet.pickle.
